In [10]:
'''
This is code for extracting NN features of face image data 
and then fit a linear model to predict social attributes of a face
Available dataset: TWIN, CHICAGO and MIT
Available NN feature: 'caffeNet','vgg16','vggFace' and 'faceSNN'

BY Linjie Li
Please run this code on guru2/neon server
'''
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os

from sklearn.decomposition import PCA as sklearnPCA

# Load image dataset#
Dataset = 'mit' # 'twin', 'chicago' or 'mit', 'zhihu' or 'funnyFace'
if Dataset == 'twin':
    imPath = '../../processing/imageProcessing/paddedImages/'
    ext = '.png'
elif Dataset == 'chicago':
    imPath = '../../ChicagoFaceDataset/CFD Version 2.0/CFD 2.0 Images/'
    ext = 'N.jpg'
elif Dataset == 'mit':
    imPath = '../../MIT2kFaceDataset/2kfaces/'
    ext = '.jpg'
elif Dataset =='funnyFace':
    imPath = '../funnyFace/'
    ext = '.png'
else:
    imPath = '../../../zhihu/'
    ext = '.jpg'
imList = []
for dirpath, dirnames, filenames in os.walk(imPath):
    for filename in [f for f in filenames if f.endswith(ext)]:
        imList.append(os.path.join(dirpath, filename))
imList.sort()
print len(imList)
#print imPath

2222


In [11]:
# Make sure that caffe is on the python path:
caffe_root = '/home/lli-ms/caffe/'
pretrained_model_root = '/home/lli-ms/caffe/'

# run this line one time only!
import sys
caffePython = pretrained_model_root + 'python'
if caffePython not in sys.path:
    sys.path.insert(0, caffePython)


import caffe
# Load mean
mu = np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy')
mu = mu.mean(1).mean(1)  # average over pixels to obtain the mean (BGR) pixel values
print 'mean-subtracted values:', zip('BGR', mu)

# Load the trained net
MODEL = 'vgg16' #'caffeNet','vgg16','vggFace' or 'faceSNN'

saveFigPath = '../Result/'+Dataset+'/'+MODEL
if not os.path.exists(saveFigPath):
    os.makedirs(saveFigPath)
    
if MODEL == 'vgg16':
    MODEL_FILE = caffe_root +'models/VGG16/VGG_ILSVRC_16_layers_deploy.prototxt'
    PRETRAINED_FILE = caffe_root + 'models/VGG16/VGG_ILSVRC_16_layers.caffemodel'
elif MODEL == 'caffeNet':
    MODEL_FILE = caffe_root + 'models/bvlc_reference_caffenet/deploy.prototxt'
    PRETRAINED_FILE = caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'
elif MODEL == 'vggFace':
    MODEL_FILE = caffe_root + 'models/VGGFACE/VGG_CNN_F_deploy.prototxt'
    PRETRAINED_FILE = caffe_root + 'models/VGGFACE/VGG_CNN_F.caffemodel'
    MEAN_FILE = caffe_root + 'models/VGGFACE/VGG_mean.binaryproto'
else:
    MODEL = 'faceSNN'
    MODEL_FILE = caffe_root +'models/sraonet/siamese_lecun_deploy.prototxt'
    PRETRAINED_FILE = caffe_root + 'models/sraonet/snapshots/sraonet_lecun_gd_sub2_iter_100000.caffemodel'
    
caffe.set_device(1)
caffe.set_mode_gpu()
if not os.path.isfile(PRETRAINED_FILE):
    print("No caffemodel!!!")
elif not os.path.isfile(MODEL_FILE):
    print("No MODEL !!!")
else:
    print "Defining the net!"
    net = caffe.Net(MODEL_FILE,
                PRETRAINED_FILE,
                caffe.TEST)
# input preprocessing: 'data' is the name of the input blob == net.inputs[0]
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
if MODEL != 'faceSNN':
    # subtract the dataset-mean value in each channel
    transformer.set_mean('data', mu)
for layer_name, param in net.params.iteritems():
    print layer_name + '\t' + str(param[0].data.shape), str(param[1].data.shape)
transformer.set_transpose('data', (2,0,1))
# the reference model operates on images in [0,255] range instead of [0,1]
transformer.set_raw_scale('data', 255) 
# the reference model has channels in BGR order instead of RGB
transformer.set_channel_swap('data', (2,1,0))

/usr/lib/python2.7/dist-packages/skimage/io/_plugins/plugin.py:47: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.
  cp.read(f)
/usr/lib/python2.7/dist-packages/skimage/io/_plugins/plugin.py:47: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.
  cp.read(f)
/usr/lib/python2.7/dist-packages/skimage/io/_plugins/plugin.py:47: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.
  cp.read(f)
/usr/lib/python2.7/dist-packages/skimage/io/_plugins/plugin.py:47: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.
  cp.read(f)
/usr/lib/python2

mean-subtracted values: [('B', 104.0069879317889), ('G', 116.66876761696767), ('R', 122.6789143406786)]
Defining the net!
conv1_1	(64, 3, 3, 3) (64,)
conv1_2	(64, 64, 3, 3) (64,)
conv2_1	(128, 64, 3, 3) (128,)
conv2_2	(128, 128, 3, 3) (128,)
conv3_1	(256, 128, 3, 3) (256,)
conv3_2	(256, 256, 3, 3) (256,)
conv3_3	(256, 256, 3, 3) (256,)
conv4_1	(512, 256, 3, 3) (512,)
conv4_2	(512, 512, 3, 3) (512,)
conv4_3	(512, 512, 3, 3) (512,)
conv5_1	(512, 512, 3, 3) (512,)
conv5_2	(512, 512, 3, 3) (512,)
conv5_3	(512, 512, 3, 3) (512,)
fc6	(4096, 25088) (4096,)
fc7	(4096, 4096) (4096,)
fc8	(1000, 4096) (1000,)


In [12]:
# read in image list 
def readFile(fName):
    text_file = open(fName, "r")
    lines = text_file.read().split('\n')
    text_file.close()
    return lines

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

if MODEL == 'vgg16' or MODEL == 'vggFace':
    imgeReshape = [224,224]
    featureLayer = 'conv5_2' 
elif MODEL == 'caffeNet':
    imgeReshape = [227,227]
    featureLayer = 'fc6'
else:
    imgeReshape = [56,46]
    featureLayer = 'fc6'
if 'fc' in featureLayer:
    featureNum = net.params[featureLayer][1].data.shape[0]
else:
    featureNum = net.blobs[featureLayer].data.flatten().shape[0]/net.blobs[featureLayer].data.shape[0]
    
if Dataset == 'twin':
    features = np.zeros([4,len(imList)/4,featureNum])
    perImNum = len(imList)/4
    img_type_num = {}
    img_type_index = {}
    img_type_list = {}
    type_index = 0
else:
    features = np.zeros([len(imList),featureNum])
print featureNum

100352


In [ ]:
totalNum = 0

# print len(imList)
for img in imList:
    imgName = os.path.basename(img)
    if imgName.endswith(('.jpg','.png')):
        input_image = caffe.io.load_image(img)
        net.blobs['data'].reshape(1,3,imgeReshape[0],imgeReshape[1])
        net.blobs['data'].data[...] = transformer.preprocess('data', input_image)
        out = net.forward()
        feat = net.blobs[featureLayer].data
        if Dataset =='twin':
            img_type = int(imgName[7:-4])/perImNum
            img_index = int(imgName[7:-4])%perImNum
            #print 'img_type:',img_type
            if img_type in img_type_num.keys():
                img_type_num[img_type] = img_type_num[img_type] + 1
                img_type_list[img_type][img_index] = img
            else:
                img_type_num[img_type] = 0
                img_type_list[img_type] = [None]*perImNum
                img_type_index[img_type] = type_index
                type_index +=1
            #print 'img_type_index:',img_type_index[img_type]
            features[img_type_index[img_type],img_type_num[img_type]] = feat.flatten()
        else:
            # need to be further revised!
            features[totalNum] = feat.flatten()
            #print features[totalNum]
        totalNum +=1
    else:
        print img
#print len(img_type_num)
print totalNum
#print img_type_list

if Dataset == 'twin':
    featureMat = np.zeros((totalNum,featureNum))
    k = 0
    for i in range(features.shape[0]):
        for j in range(features[i].shape[0]):
            if sum(features[i,j,:])!=0:
                featureMat[k,:] = features[i,j,:]
                k +=1
else:
    featureMat = features
#print featureMat

In [ ]:
if MODEL != 'faceSNN':
    explained_variance = 0.90
else:
    explained_variance = featureNum

sklearn_pca = sklearnPCA(n_components=explained_variance, whiten  = True)
feature_transf = sklearn_pca.fit_transform(featureMat)
print 'The number of PCs needed to retain %.3f variance is %d.' \
      % (explained_variance, feature_transf.shape[1])

In [ ]:
np.savetxt('/home/lli-ms/features/'+MODEL+'_'+featureLayer+'_'+Dataset+\
           '_feature_pca.csv', feature_transf, delimiter=',')

In [4]:
import numpy as np
MODEL = 'vgg16'
Dataset = 'mit'
featureLayer = 'conv5_2'
feature_transf = np.loadtxt('/home/lli-ms/features/'+MODEL+'_'+featureLayer+'_'+\
                            Dataset+'_feature_pca.csv', delimiter=',')

In [1]:
import sys
#print sys.path
# local
# PkgPath = '/Users/Olivialinlin/Documents/Github/attractiveness_datamining/linjieCode/code'
# server
PkgPath = '/home/lli-ms/attractiveness_datamining/linjieCode/code'

if PkgPath not in sys.path:
    sys.path.insert(0, PkgPath)
#print sys.path
from xVal_train_test import Train_Test
import pandas as pd
import sklearn
import numpy as np

In [2]:
socialMeasures = '../Result/mit/socialMeasures.csv'
socialMeasures = pd.read_csv(socialMeasures,index_col = 0)
socialAttr = socialMeasures.columns.tolist()
delElement = ['subage.1', 'submale.1', 'subrace.1','subage', 'submale',\
              'subrace','catch', 'catchAns','catch.1','catchAns.1']
social2Attr = [x for x in socialAttr if x not in delElement]
socialMeasuresClean = socialMeasures.loc[:,social2Attr].as_matrix()
print socialMeasuresClean
np.savetxt('../Result/mit/socialMeasuresClean.csv', socialMeasuresClean, delimiter=',')
# print socialMeasures
# socialMeasures = pd.read_csv(socialMeasures,index_col = 0)

[[ 4.285714  4.2       5.933333 ...,  3.666667  2.933333  2.266667]
 [ 3.333333  4.733333  5.866667 ...,  4.333333  3.2       3.666667]
 [ 2.666667  3.866667  5.933333 ...,  3.6       3.8       3.666667]
 ..., 
 [ 4.066667  5.        6.066667 ...,  3.        2.666667  3.866667]
 [ 4.4       5.066667  5.066667 ...,  4.066667  3.066667  5.4     ]
 [ 3.8       3.533333  5.333333 ...,  4.533333  3.733333  3.8     ]]


In [6]:
modelList = []
optFeaNumList = []
for attr in social2Attr:
    print attr
    mean_rating = socialMeasures.loc[:,attr].tolist()
    mean_rating = map(float, mean_rating)
    mean_rating = np.array(mean_rating)
    baseLine = mean_rating.mean()
    print 'mean rating: ', baseLine

    predictionModel = sklearn.linear_model.RidgeCV(alphas=np.logspace(-3,2,num=20), fit_intercept=True)
    myModel,optFeaNum = Train_Test(mean_rating, feature_transf,xVal = True,\
                                            pModel = predictionModel,getMaxMin = False,\
                                           numTrain = 1,savePath = '../Result/'+Dataset,\
                                             MODEL= MODEL, plotPredActual = False,returnModel = True)
    modelList.append(myModel)
    optFeaNumList.append(optFeaNum)

atypical
mean rating:  4.08832261206
**************************Result of train and test**************************************
number of features: 68
On validation set:
Residual sum of squares: 0.38
Variance score is: 0.17
Correlation between predicted ratings and actual ratings is: 0.4164
 
On training set:
Residual sum of squares: 0.35
Variance score is: 0.25
Correlation between predicted ratings and actual ratings is: 0.5053
****************************************************************************************
boring
mean rating:  4.29054390414
**************************Result of train and test**************************************
number of features: 68
On validation set:
Residual sum of squares: 0.43
Variance score is: 0.36
Correlation between predicted ratings and actual ratings is: 0.5989
 
On training set:
Residual sum of squares: 0.41
Variance score is: 0.40
Correlation between predicted ratings and actual ratings is: 0.6356
***************************************************

In [33]:
predictRatingAll = np.zeros(socialMeasuresClean.shape)
for i in range(len(modelList)):
    m = modelList[i]
    num = optFeaNumList[i]
    featureOpt = feature_transf[:,:num]
    #print featureOpt.shape
    predictRatingAll[:,i] = m.predict(featureOpt)
np.savetxt('/home/lli-ms/features/'+MODEL+'_'+featureLayer+'_'+Dataset+\
           '_predict_ratings.csv', predictRatingAll, delimiter=',')
correlationAll = np.corrcoef(predictRatingAll.T)
np.savetxt('predictCorrelation.csv', correlationAll, delimiter=',')

In [22]:
%matplotlib inline
from plotFunc import plotHeatMap
import pandas as pd
p = pd.read_csv('./correlation_array', index_col = 0)
column_name = p.columns.tolist()
data = p.as_matrix()
print column_name
plotHeatMap(data,clusterNum= 15,xTickLabel=column_name,\
            colorMapName='coolwarm',figName = '',fSize = 3.5\
            ,dendro = False)

['atypical', 'boring', 'calm', 'cold', 'common', 'confident', 'egotistic', 'emotUnstable', 'forgettable', 'intelligent', 'introverted', 'kind', 'responsible', 'trustworthy', 'unattractive', 'unemotional', 'unfamiliar', 'unfriendly', 'unhappy', 'weird', 'aggressive', 'attractive', 'caring', 'emotStable', 'emotional', 'familiar', 'friendly', 'happy', 'humble', 'interesting', 'irresponsible', 'mean', 'memorable', 'normal', 'sociable', 'typical', 'uncertain', 'uncommon', 'unintelligent', 'untrustworthy']
Into plotting...
[39 20  3  7 10 15 17 18 38 19 30 31 36 33 27 26 13 23 22 34 11  9 29  5 21
  2 12  1 14 16  0 35 28  8  6 32  4 37 24 25]
[2, 2, 3, 13, 1, 3, 1, 13, 1, 3, 13, 8, 3, 8, 2, 13, 2, 13, 13, 13, 13, 3, 8, 8, 1, 1, 8, 8, 1, 3, 13, 13, 1, 8, 8, 1, 13, 1, 13, 13]
[ 4  1 10  3  7 12  6  3  2 10  3  9 10  9  1  3  4  3  3  3  3 12  9  9 14
 13  9  9 11 12  3  3 15  9  9  8  3  5  3  3]
Finish dendrogram...
Start plotting heatmap...


(array([39, 20,  3,  7, 10, 15, 17, 18, 38, 19, 30, 31, 36, 33, 27, 26, 13,
        23, 22, 34, 11,  9, 29,  5, 21,  2, 12,  1, 14, 16,  0, 35, 28,  8,
         6, 32,  4, 37, 24, 25]),
 array([ 4,  1, 10,  3,  7, 12,  6,  3,  2, 10,  3,  9, 10,  9,  1,  3,  4,
         3,  3,  3,  3, 12,  9,  9, 14, 13,  9,  9, 11, 12,  3,  3, 15,  9,
         9,  8,  3,  5,  3,  3], dtype=int32))

RuntimeError: LaTeX was not able to process the following string:
'lp'
Here is the full report generated by LaTeX: 

